In [ ]:
import pandas as pd
import numpy as np
from tmdbv3api import Movie, TMDb
from flask import Flask
from flask import request, redirect

app = Flask(__name__)

@app.route("/search")
def search_reco():
    movie = Movie()
    tmdb = TMDb()
    tmdb.api_key = 'c78f6d25f134b905f3df83accbce966c'
    tmdb.language = 'ko-KR'

    df1 = pd.read_csv('tmdb_5000_credits.csv')
    df2 = pd.read_csv('tmdb_5000_movies.csv')
    
    df1.columns = ['id', 'title', 'cast', 'crew']
    
    df1 = pd.read_csv('tmdb_5000_credits.csv')
    df2 = pd.read_csv('tmdb_5000_movies.csv')
    
#     df2 = df2.merge(df1[['id', 'cast', 'crew']], on='id')
    
#     C = df2['vote_average'].mean()
    
#     m = df2['vote_count'].quantile(0.9)
    
#     q_movies = df2.copy().loc[df2['vote_count'] >= m]
    
#     def weighted_rating(x, m=m, C=C):
#         v = x['vote_count']
#         R = x['vote_average']
#         q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
#         return (v / (v + m) * R) + (m / (m + v) * C)
        
#     q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
    
#     q_movies = q_movies.sort_values('score', ascending=False)
    
#     ---------------------------------------------------------------
    
#     from sklearn.feature_extraction.text import TfidfVectorizer
#     tfidf = TfidfVectorizer(stop_words='english')
    
#     from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
    
#     df2['overview'] = df2['overview'].fillna('')
    
#     tfidf_matrix = tfidf.fit_transform(df2['overview'])
    
#     from sklearn.metrics.pairwise import linear_kernel

#     cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
#     indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()
    #------------------------------------------------------------------------------------
    
    # 영화의 제목을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 영화 목록 반환
    def get_recommendations(title, cosine_sim=cosine_sim):
        # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기
        idx = indices[title]
    
        # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 코사인 유사도 기준으로 내림차순 정렬
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱
        sim_scores = sim_scores[0:10]

        # 추천 영화 목록 10개의 인덱스 정보 추출
        movie_indices = [i[0] for i in sim_scores]

        # 인덱스 정보를 통해 영화 제목 추출
        return df2['title'].iloc[movie_indices]
    
#     test_idx = indices['The Dark Knight Rises'] # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기
    
#     test_sim_scores = list(enumerate(cosine_sim[3])) # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
    
#     test_sim_scores = sorted(test_sim_scores, key=lambda x: x[1], reverse=True) # 코사인 유사도 기준으로 내림차순 정렬
    
    def get_second(x):
        return x[1]

    lst = ['인덱스', '유사도']
    
    # 추천 영화 목록 10개의 인덱스 정보 추출
    test_movie_indices = [i[0] for i in test_sim_scores[1:11]]
    
    s1 = [{"id": 28, "name": "Action"}]
    s2 = '[{"id": 28, "name": "Action"}]
    
    from ast import literal_eval
    s2 = literal_eval(s2)
    s2, type(s2)
    
    features = ['cast', 'crew', 'keywords', 'genres']
    for feature in features:
        df2[feature] = df2[feature].apply(literal_eval)
        
    # 감독 정보를 추출
    def get_director(x):
        for i in x:
            if i['job'] == 'Director':
                return i['name']
            return np.nan
        
    df2['director'] = df2['crew'].apply(get_director)
    
    # 처음 3개의 데이터 중에서 name 에 해당하는 value 만 추출
    def get_list(x):
        if isinstance(x, list):
            names = [i['name'] for i in x]
            if len(names) > 3:
                names = names[:3]
            return names
        return []
    
    features = ['cast', 'keywords', 'genres']
    for feature in features:
        df2[feature] = df2[feature].apply(get_list)
        
    def clean_data(x):
        if isinstance(x, list):
            return [str.lower(i.replace(' ', '')) for i in x]
        else:
            if isinstance(x, str):
                return str.lower(x.replace(' ', ''))
            else:
                return ''
            
    features = ['cast', 'keywords', 'director', 'genres']
    for feature in features:
        df2[feature] = df2[feature].apply(clean_data)
        
    def create_soup(x):
        return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
    df2['soup'] = df2.apply(create_soup, axis=1)
    
    from sklearn.feature_extraction.text import CountVectorizer

    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(df2['soup'])
    
    from sklearn.metrics.pairwise import cosine_similarity
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
    
    df2 = df2.reset_index()
    indices = pd.Series(df2.index, index=df2['title'])
    
#     import pickle
    
#     movies = df2[['id', 'title']].copy()
    
#     pickle.dump(movies, open('movies.pickle', 'wb'))
    
#     pickle.dump(cosine_sim2, open('cosine_sim.pickle', 'wb'))
    
    search_title = request.values.get("search_title")
    result = get_recommendations(serch_title, cosine_sim2)
    
    result_image = []
    result_title = []
    for i in movie_indices:
        id = movies['id'].iloc[i]
        details = movie.details(id)
        
        image_path = details['poster_path']
        if image_path:
            image_path = 'https://image.tmdb.org/t/p/w500' + image_path
        else:
            image_path = 'no_image.jpg'
            result_image.append(image_path)
            result_title.append(details['title'])
        
    return redirect("http://localhost:8081/SuitMV_v1/SearchCon.do.java?result_image="+result_image+"&result_title="+result_title)

if __name__ == '__main__':
    app.run(host='localhost', port='9000')

In [1]:
pip install tmdbv3api

  Using cached tmdbv3api-1.7.7-py2.py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tmdbv3api import Movie, TMDb